In [2]:
import numpy as np
import scipy.io as sio
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score
import spectral
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os

# 用于测试样本的比例
test_ratio = 0.90
# 每个像素周围提取 patch 的尺寸
patch_size = 15
name = "IP"
if name == 'PU':
    class_num = 9
if name == 'SA':
    class_num = 16
if name == 'IP':
    class_num = 16
################################下载数据集################################
#wget.download('http://www.ehu.eus/ccwintco/uploads/6/67/Indian_pines_corrected.mat', 'Indian_pines_corrected.mat')
#wget.download('http://www.ehu.eus/ccwintco/uploads/c/c4/Indian_pines_gt.mat', 'Indian_pines_gt.mat')

################################(class)模型的具体操作步骤################################

class CNN_1D(nn.Module):
    """
    Based on Paper:Chen, Y. Deep Feature Extraction and Classification of Hyperspectral Images Based on Convolutional Neural Networks. TGRS
    input shape:[N,C=1,L=spectral_channel]
    """
    def __init__(self):
        super(CNN_1D, self).__init__()
        #self.dataset = dataset
        self.C2 = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=6, kernel_size=(3,)),
            nn.BatchNorm1d(6),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=2, stride=2),
        )
        self.C4 = nn.Sequential(
            nn.Conv1d(in_channels=6, out_channels=12, kernel_size=(3,)),
            nn.BatchNorm1d(12),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.MaxPool1d(kernel_size=2, stride=2),
        )
        self.C6 = nn.Sequential(
            nn.Conv1d(in_channels=12, out_channels=24, kernel_size=(3,)),
            nn.BatchNorm1d(24),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.MaxPool1d(kernel_size=2, stride=2),
        )
        self.C8 = nn.Sequential(
            nn.Conv1d(in_channels=24, out_channels=48, kernel_size=(3,)),
            nn.BatchNorm1d(48),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),  # for IP ,p=0.3, else(PU and SV) is 0.1
            nn.MaxPool1d(kernel_size=2, stride=2),
        )
        self.fc = nn.Sequential(
            nn.Linear(21504,128),
            nn.Dropout(p=0.4),
        )
        self.classifier = nn.Linear(128,class_num)

    def forward(self, x):
       # print(x.shape)
        #x = torch.squeeze(x, dim=2).transpose(1, 2)
        x = x.reshape(x.shape[0],1,x.shape[1]*x.shape[2]*x.shape[3])
       # print(x.shape)
        C2 = self.C2(x)
        #print(C2.shape)
        C4 = self.C4(C2)
        #print(C4.shape)
        last = self.C6(C4)
        #print(last.shape)
        last = self.C8(last)
        #print(last.shape)
        last = torch.reshape(last, (last.shape[0], -1))
        #print(last.shape)
        FC = self.fc(last)
        #print(FC.shape)
        out = self.classifier(FC)
        return out


################################数据处理################################
# 对高光谱数据 X 应用 PCA 变换
def applyPCA(X, numComponents):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=numComponents, whiten=True)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0], X.shape[1], numComponents))
    return newX

# 对单个像素周围提取 patch 时，边缘像素就无法取了，因此，给这部分像素进行 padding 操作
def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

# # 在每个像素周围提取 patch ，然后创建成符合 keras 处理的格式
# def createImageCubes(X, y, windowSize=5, removeZeroLabels = True):#X_pca, y = createImageCubes(X_pca, y, windowSize=patch_size)
#     # 给 X 做 padding
#     margin = int((windowSize - 1) / 2)
#     zeroPaddedX = padWithZeros(X, margin=margin)
#     # split patches
#     patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))##(145*145，25，25，30)
#     patchesLabels = np.zeros((X.shape[0] * X.shape[1]))#(145*145)
#     patchIndex = 0
#     for r in range(margin, zeroPaddedX.shape[0] - margin):
#         for c in range(margin, zeroPaddedX.shape[1] - margin):
#             patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]
#             patchesData[patchIndex, :, :, :] = patch
#             patchesLabels[patchIndex] = y[r-margin, c-margin]
#             patchIndex = patchIndex + 1
#     if removeZeroLabels:
#         patchesData = patchesData[patchesLabels > 0, :, :, :]
#         patchesLabels = patchesLabels[patchesLabels > 0]
#         patchesLabels -= 1
#     return patchesData, patchesLabels
# 在每个像素周围提取 patch
def createImageCubes(X, y, windowSize=5, removeZeroLabels = True):
    # 给 X 做 padding
    margin = int((windowSize - 1) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    # 获得 y 中的标记样本数
    count = 0
    for r in range(0, y.shape[0]):
        for c in range(0, y.shape[1]):
            if y[r, c] != 0:
                count = count+1

    # split patches
    patchesData = np.zeros([count, windowSize, windowSize, X.shape[2]])
    patchesLabels = np.zeros(count)

    count = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            if y[r-margin, c-margin] != 0:
                patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]
                patchesData[count, :, :, :] = patch
                patchesLabels[count] = y[r-margin, c-margin]
                count = count + 1

    return patchesData, patchesLabels

def splitTrainTestSet(X, y, testRatio, randomState=345):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testRatio, random_state=randomState, stratify=y)
    return X_train, X_test, y_train, y_test
#类别
#class_num = 16

#X = sio.loadmat('C:\AI\data\Indian_pines_corrected.mat')['indian_pines_corrected']#X是数据集
#y = sio.loadmat('C:\AI\data\Indian_pines_gt.mat')['indian_pines_gt']#y是标签

def loadData(name):
    data_path = os.path.join(os.getcwd(), 'mnt')
    if name == 'IP':
        data = sio.loadmat(os.path.join(data_path, 'Indian_pines_corrected.mat'))['indian_pines_corrected']
        labels = sio.loadmat(os.path.join(data_path, 'Indian_pines_gt.mat'))['indian_pines_gt']
    elif name == 'SA':
        data = sio.loadmat(os.path.join(data_path, 'Salinas_corrected.mat'))['salinas_corrected']
        labels = sio.loadmat(os.path.join(data_path, 'Salinas_gt.mat'))['salinas_gt']
    elif name == 'PU':
        data = sio.loadmat(os.path.join(data_path, 'PaviaU.mat'))['paviaU']
        labels = sio.loadmat(os.path.join(data_path, 'PaviaU_gt.mat'))['paviaU_gt']
    else:
        print("NO DATASET")
        exit()
    return data, labels

X, y = loadData(name)


# 使用 PCA 降维，得到主成分的数量
if name == 'PU':
    pca_components = 32
if name == 'SA':
    pca_components = 32
if name == 'IP':
    pca_components = 32
#pca_components = 15

print('Hyperspectral data shape: ', X.shape)##(145,145,200)
print('Label shape: ', y.shape)#(145,145)

print('\n... ... PCA tranformation ... ...')

X_pca = applyPCA(X, numComponents=pca_components)
print('Data shape after PCA: ', X_pca.shape)#(145,145,30)

print('\n... ... create data cubes ... ...')
X_pca, y = createImageCubes(X_pca, y, windowSize=patch_size)#(10249,25,25,30)    (10249)
print('Data cube X shape: ', X_pca.shape)#(10249,25,25,30)
print('Data cube y shape: ', y.shape)#(10249)

print('\n... ... create train & test data ... ...')
Xtrain, Xtest, ytrain, ytest = splitTrainTestSet(X_pca, y, test_ratio)
print('Xtrain shape: ', Xtrain.shape)#(1024, 25, 25, 30)
print('Xtest  shape: ', Xtest.shape)#(9225, 25, 25, 30)

# 改变 Xtrain, Ytrain 的形状，以符合 keras 的要求
Xtrain = Xtrain.reshape(-1, pca_components, patch_size, patch_size)
Xtest  = Xtest.reshape(-1, pca_components, patch_size, patch_size)
print('before transpose: Xtrain shape: ', Xtrain.shape)#(1024, 25, 25, 30, 1)
print('before transpose: Xtest  shape: ', Xtest.shape)#(9225, 25, 25, 30, 1)

# # 为了适应 pytorch 结构，数据要做 transpose
# Xtrain = Xtrain.transpose(0, 4, 3, 1, 2)
# Xtest  = Xtest.transpose(0, 4, 3, 1, 2)
# print('after transpose: Xtrain shape: ', Xtrain.shape)#(1024, 1, 30, 25, 25)
# print('after transpose: Xtest  shape: ', Xtest.shape)#(9225, 1, 30, 25, 25)


""" Training dataset"""
class TrainDS(torch.utils.data.Dataset):
    def __init__(self):
        self.len = Xtrain.shape[0]
        self.x_data = torch.FloatTensor(Xtrain)
        self.y_data = torch.LongTensor(ytrain)
    def __getitem__(self, index):
        # 根据索引返回数据和对应的标签
        return self.x_data[index], self.y_data[index]
    def __len__(self):
        # 返回文件数据的数目
        return self.len

""" Testing dataset"""
class TestDS(torch.utils.data.Dataset):
    def __init__(self):
        self.len = Xtest.shape[0]
        self.x_data = torch.FloatTensor(Xtest)
        self.y_data = torch.LongTensor(ytest)
    def __getitem__(self, index):
        # 根据索引返回数据和对应的标签
        return self.x_data[index], self.y_data[index]
    def __len__(self):
        # 返回文件数据的数目
        return self.len

# 创建 trainloader 和 testloader
trainset = TrainDS()
testset  = TestDS()
train_loader = torch.utils.data.DataLoader(dataset=trainset, batch_size=128, shuffle=True, num_workers=0)
test_loader = torch.utils.data.DataLoader(dataset=testset,  batch_size=128, shuffle=False, num_workers=0)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
one = torch.ones(128, dtype=torch.long).to(device)
two = torch.ones(76, dtype=torch.long).to(device)
# 网络放到GPU上
net = CNN_1D().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

#开始训练
net.train()
total_loss = 0
for epoch in range(150):
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        #print(labels.shape)
        try:
            labels = labels - one
        except:
            labels = labels - two

        #print(labels)
        # 优化器梯度归零
        optimizer.zero_grad()
        # 正向传播 +　反向传播 + 优化
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print('[Epoch: %d]   [loss avg: %.4f]   [current loss: %.4f]' %(epoch + 1, total_loss/(epoch+1), loss.item()))

print('Finished Training')
path = 'model' + '.pth'
torch.save(net, path)

if name == 'PU':
    t = int(42776 * test_ratio)+1   #10249 42776 54129
if name == 'SA':
    t = int(54129 * test_ratio) + 1  # 10249 42776 54129
if name == 'IP':
    t = int(10249 * test_ratio) + 1  # 10249 42776 54129
#print(t)
a = np.ones(t)##9225是Xtest.shape[0]
ytest = ytest - a

count = 0
# 模型测试
net.eval()
for inputs, _ in test_loader:
    inputs = inputs.to(device)
    outputs = net(inputs)
    outputs = np.argmax(outputs.detach().cpu().numpy(), axis=1)
    if count == 0:
        y_pred_test =  outputs
        count = 1
    else:
        y_pred_test = np.concatenate((y_pred_test, outputs))

# 生成分类报告
classification = classification_report(ytest, y_pred_test, digits=6)
print(classification)

from operator import truediv


def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc


def reports(test_loader, y_test, name):
    count = 0
    # 模型测试
    for inputs, _ in test_loader:
        inputs = inputs.to(device)
        outputs = net(inputs)
        outputs = np.argmax(outputs.detach().cpu().numpy(), axis=1)
        if count == 0:
            y_pred = outputs
            count = 1
        else:
            y_pred = np.concatenate((y_pred, outputs))

    if name == 'IP':
        target_names = ['Alfalfa', 'Corn-notill', 'Corn-mintill', 'Corn'
            , 'Grass-pasture', 'Grass-trees', 'Grass-pasture-mowed',
                        'Hay-windrowed', 'Oats', 'Soybean-notill', 'Soybean-mintill',
                        'Soybean-clean', 'Wheat', 'Woods', 'Buildings-Grass-Trees-Drives',
                        'Stone-Steel-Towers']
    elif name == 'SA':
        target_names = ['Brocoli_green_weeds_1', 'Brocoli_green_weeds_2', 'Fallow', 'Fallow_rough_plow',
                        'Fallow_smooth',
                        'Stubble', 'Celery', 'Grapes_untrained', 'Soil_vinyard_develop', 'Corn_senesced_green_weeds',
                        'Lettuce_romaine_4wk', 'Lettuce_romaine_5wk', 'Lettuce_romaine_6wk', 'Lettuce_romaine_7wk',
                        'Vinyard_untrained', 'Vinyard_vertical_trellis']
    elif name == 'PU':
        target_names = ['Asphalt', 'Meadows', 'Gravel', 'Trees', 'Painted metal sheets', 'Bare Soil', 'Bitumen',
                        'Self-Blocking Bricks', 'Shadows']

    classification = classification_report(y_test, y_pred, target_names=target_names)
    oa = accuracy_score(y_test, y_pred)
    confusion = confusion_matrix(y_test, y_pred)
    each_acc, aa = AA_andEachClassAccuracy(confusion)
    kappa = cohen_kappa_score(y_test, y_pred)

    return classification, confusion, oa * 100, each_acc * 100, aa * 100, kappa * 100


# 将结果写在文件里
classification, confusion, oa, each_acc, aa, kappa = reports(test_loader, ytest, name)
classification = str(classification)
confusion = str(confusion)
file_name = "classification_report.txt"

# with open(file_name, 'w') as x_file:
#     x_file.write('\n')
#     x_file.write('{} Kappa accuracy (%)'.format(kappa))
#     x_file.write('\n')
#     x_file.write('{} Overall accuracy (%)'.format(oa))
#     x_file.write('\n')
#     x_file.write('{} Average accuracy (%)'.format(aa))
#     x_file.write('\n')
#     x_file.write('\n')
#     x_file.write('{}'.format(classification))
#     x_file.write('\n')
#     x_file.write('{}'.format(confusion))
# 显示结果
# load the original image


# X, y = loadData(name)
# height = y.shape[0]
# width = y.shape[1]

# X = applyPCA(X, numComponents=pca_components)
# X = padWithZeros(X, patch_size // 2)

# # 逐像素预测类别
# outputs = np.zeros((height, width))
# for i in range(height):
#     for j in range(width):
#         if int(y[i, j]) == 0:
#             continue
#         else:
#             image_patch = X[i:i + patch_size, j:j + patch_size, :]
#             #print(image_patch.shape)
#             image_patch = image_patch.reshape(1, image_patch.shape[0], image_patch.shape[1], image_patch.shape[2])
#             #print(image_patch.shape)
#             image_patch = image_patch.reshape(1, pca_components, patch_size, patch_size)
#             X_test_image = torch.FloatTensor(image_patch).to(device)
#             #print(X_test_image.shape)
#             prediction = net(X_test_image)
#             prediction = np.argmax(prediction.detach().cpu().numpy(), axis=1)
#             outputs[i][j] = prediction + 1
#     if i % 20 == 0:
#         print('... ... row ', i, ' handling ... ...')

# oringal_image = spectral.imshow(classes=y, figsize=(7, 7))
# predict_image = spectral.imshow(classes=outputs.astype(int), figsize=(7, 7))
# spectral.save_rgb(name + "2D-原始.jpg", y.astype(int), colors=spectral.spy_colors)
# spectral.save_rgb(name + "2D-预测.jpg", outputs.astype(int), colors=spectral.spy_colors)

Hyperspectral data shape:  (145, 145, 200)
Label shape:  (145, 145)

... ... PCA tranformation ... ...
Data shape after PCA:  (145, 145, 32)

... ... create data cubes ... ...
Data cube X shape:  (10249, 15, 15, 32)
Data cube y shape:  (10249,)

... ... create train & test data ... ...
Xtrain shape:  (1024, 15, 15, 32)
Xtest  shape:  (9225, 15, 15, 32)
before transpose: Xtrain shape:  (1024, 32, 15, 15)
before transpose: Xtest  shape:  (9225, 32, 15, 15)
[Epoch: 1]   [loss avg: 34.7228]   [current loss: 3.9795]
[Epoch: 2]   [loss avg: 28.0304]   [current loss: 2.0274]
[Epoch: 3]   [loss avg: 24.4079]   [current loss: 1.9864]
[Epoch: 4]   [loss avg: 21.4352]   [current loss: 1.2495]
[Epoch: 5]   [loss avg: 19.2614]   [current loss: 1.2514]
[Epoch: 6]   [loss avg: 17.6774]   [current loss: 1.2229]
[Epoch: 7]   [loss avg: 16.3597]   [current loss: 0.8708]
[Epoch: 8]   [loss avg: 15.1696]   [current loss: 0.8303]
[Epoch: 9]   [loss avg: 14.1721]   [current loss: 0.6941]
[Epoch: 10]   [loss

/root/miniconda3/envs/myconda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/miniconda3/envs/myconda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/miniconda3/envs/myconda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0   1.000000  0.536585  0.698413        41
         1.0   0.719882  0.757977  0.738438      1285
         2.0   0.813296  0.769746  0.790922       747
         3.0   0.987013  0.356808  0.524138       213
         4.0   0.792000  0.910345  0.847059       435
         5.0   0.917614  0.983257  0.949302       657
         6.0   1.000000  0.040000  0.076923        25
         7.0   0.995360  0.997674  0.996516       430
         8.0   0.000000  0.000000  0.000000        18
         9.0   0.711520  0.840000  0.770440       875
        10.0   0.848382  0.901357  0.874068      2210
        11.0   0.784314  0.599251  0.679406       534
        12.0   0.935484  0.940541  0.938005       185
        13.0   0.951407  0.979807  0.965398      1139
        14.0   0.872951  0.613833  0.720812       347
        15.0   0.947368  0.428571  0.590164        84

    accuracy                       0.835230      9225
   macro avg   0.829787  0

/root/miniconda3/envs/myconda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/miniconda3/envs/myconda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/miniconda3/envs/myconda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
